# Stacking and unstacking data

In [1]:
import pandas as pd
from dfply import *

## Reshaping data

Two ways

* We can **stack** data into a *tall* format.
* We can **unstack** data into a *long* format.

## (totally real and not at all made-up) Example - Quarterly Auto Sales

**Note** the last four columns are

* same measurements
* same units

In [2]:
sales = pd.read_csv("./data/auto_sales.csv")
sales

,Salesperson,Compact,Sedan,SUV,Truck
0,Ann,22,18,15.0,12
1,Bob,19,12,17.0,20
2,Doug,20,13,NaN,20
3,Yolanda,19,8,32.0,15
4,Xerxes,12,23,18.0,9


## Stacking measurements of the same type/units

<img src="./img/stack_in_action.gif" width=600>

We can fix issues with informative column labels by stacking the data with `gather`

## A Stack by any other name ...

The act of stacking similar columns goes by various names.

* JMP and Minitab call this *stack*
* `pandas` calls this *melt*
* Wickham/`tidyr`/`dfply` call this *gather*

I prefer **stack**, primarily because it makes it clear we are *melting*/*gathering* data vertically.

## Stacking data in `pandas` with `gather`

Syntax: `gather(lbl_col_name, val_col_name, cols_to_stack)`

In [3]:
sales_cols = ['Compact', 'Sedan', 'SUV', 'Truck']
sales_stacked = (sales 
                 >> gather("CarType","QrtSales", sales_cols))
sales_stacked >> head

,Salesperson,CarType,QrtSales
0,Ann,Compact,22.0
1,Bob,Compact,19.0
2,Doug,Compact,20.0
3,Yolanda,Compact,19.0
4,Xerxes,Compact,12.0


## Unstacking Data with `spread`

Syntax: `spread(split_by_col, to_split_col)`

In [4]:
(sales_stacked
 >> spread(X.CarType, X.QrtSales))

,Salesperson,Compact,SUV,Sedan,Truck
0,Ann,22.0,15.0,18.0,12.0
1,Bob,19.0,17.0,12.0,20.0
2,Doug,20.0,NaN,13.0,20.0
3,Xerxes,12.0,18.0,23.0,9.0
4,Yolanda,19.0,32.0,8.0,15.0


## Safely working with `gather` and `spread`


We were lucky the last example worked.  Note that 

* `spread` needs a unique column to work properly.  
* `gather` will add a column by setting `add_id=True`

In [5]:
sales_stacked = sales >> gather("CarType","QrtSales", sales_cols, add_id=True)
sales_stacked >> head(2)

,Salesperson,_ID,CarType,QrtSales
0,Ann,0,Compact,22.0
1,Bob,1,Compact,19.0


In [7]:
sales_stacked >> spread(X.CarType, X.QrtSales) >> head(2)

,Salesperson,_ID,Compact,SUV,Sedan,Truck
0,Ann,0,22.0,15.0,18.0,12.0
1,Bob,1,19.0,17.0,12.0,20.0


## Why Stack?

* Perform transformations on many columns.
* Fix problems with the Golden Rule

## Example - Switching Units on All Sales

Suppose your manager wants these numbers in *monthly* sales.  You could

1. Adjust each column with a separate formula
2. Stack --> Transform once --> Unstack

#### Method 1 - Column Transformations

In [6]:
(sales
 >> mutate(Compact = X.Compact/3,
           SUV =   X.SUV/3,
           Sedan = X.Sedan/3,
           Truck = X.Truck/3)
 >> head(2))

,Salesperson,Compact,Sedan,SUV,Truck
0,Ann,7.333333,6.0,5.000000,4.000000
1,Bob,6.333333,4.0,5.666667,6.666667


#### Method 2 - Stack-Transform-Unstack

In [7]:
(sales 
 >> gather("CarType","QrtSales", sales_cols)
 >> mutate(MonSales = X.QrtSales/3)
 >> drop(X.QrtSales)
 >> spread(X.CarType, X.MonSales)
 >> head(2))

,Salesperson,Compact,SUV,Sedan,Truck
0,Ann,7.333333,5.000000,6.0,4.000000
1,Bob,6.333333,5.666667,4.0,6.666667


## Comparing the two methods

**Method 1:**
* More straight forward
* Lots of repeated code
* Doesn't scale ... imagine 100+ columns

**Method 2:**
* More complicated
* Scales well


## <font color="red"> Exercise 1 </font>
    
**Task:** Load the `health_survey.csv` data and use the Stack-Transform-Unstack trick to transform the responses to a Lickert scale where *Strongly Agree* mapped to 5 and *Strongly Disagree* mapped to 1


In [20]:
survey = pd.read_csv("./data/health_survey.csv")
survey.head(2)

,Unnamed: 0,F1,F5,F2,F1.1,F2.1,F6,F4,F3,F5.1,...,F2.9,F3.4,F4.3,F2.10,F1.7,F6.4,F4.4,F5.7,F3.5,F2.11
0,1,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,...,Somewhat Agree,Somewhat Disagree,Neither Agree nor Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Agree
1,2,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Somewhat Agree,Neither Agree nor Disagree,Neither Agree nor Disagree,...,Somewhat Agree,Somewhat Agree,Neither Agree nor Disagree,Somewhat Agree,Somewhat Agree,Somewhat Disagree,Neither Agree nor Disagree,Somewhat Agree,Neither Agree nor Disagree,Somewhat Agree


In [29]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264 entries, 0 to 263
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  264 non-null    int64 
 1   F1          264 non-null    object
 2   F5          261 non-null    object
 3   F2          262 non-null    object
 4   F1.1        264 non-null    object
 5   F2.1        263 non-null    object
 6   F6          264 non-null    object
 7   F4          264 non-null    object
 8   F3          263 non-null    object
 9   F5.1        264 non-null    object
 10  F1.2        264 non-null    object
 11  F2.2        264 non-null    object
 12  F6.1        263 non-null    object
 13  F2.3        264 non-null    object
 14  F4.1        264 non-null    object
 15  F2.4        262 non-null    object
 16  F5.2        263 non-null    object
 17  F2.5        264 non-null    object
 18  F6.2        264 non-null    object
 19  F1.3        264 non-null    object
 20  F2.6      

In [14]:
survey.columns[1:]

Index(['F1', 'F5', 'F2', 'F1.1', 'F2.1', 'F6', 'F4', 'F3', 'F5.1', 'F1.2',
       'F2.2', 'F6.1', 'F2.3', 'F4.1', 'F2.4', 'F5.2', 'F2.5', 'F6.2', 'F1.3',
       'F2.6', 'F5.3', 'F4.2', 'F2.7', 'F3.1', 'F2.8', 'F5.4', 'F3.2', 'F1.4',
       'F3.3', 'F1.5', 'F5.5', 'F6.3', 'F1.6', 'F5.6', 'F2.9', 'F3.4', 'F4.3',
       'F2.10', 'F1.7', 'F6.4', 'F4.4', 'F5.7', 'F3.5', 'F2.11'],
      dtype='object')

In [16]:
survey_cols = survey.columns[1:]


In [40]:
survey_gather = (survey
                >> rename(User = X["Unnamed: 0"])
                >> gather("Response","Rating", survey_cols))

survey_gather

,User,Response,Rating
0,1,F1,Somewhat Agree
1,2,F1,Somewhat Agree
2,3,F1,Strongly Agree
3,4,F1,Somewhat Agree
4,5,F1,Strongly Agree
...,...,...,...
11611,260,F2.11,Neither Agree nor Disagree
11612,261,F2.11,Somewhat Agree
11613,262,F2.11,Somewhat Agree
11614,263,F2.11,Somewhat Agree


In [48]:
survey_gather = (survey
                >> rename(User = X["Unnamed: 0"])
                >> gather("Response","Rating", survey_cols)
                #>> filter_by(X.User == 1))
                >> mutate(Lickert = case_when(
                    #(X["Country2"].eq("Guadeloupe") == True, "France"),
                    (X["Rating"] == "Strongly Agree", 5),
                    (X["Rating"] == "Somewhat Agree", 4),
                    (X["Rating"] == "Neither Agree nor Disagree", 3),
                    (X["Rating"] == "Somewhat Disagree", 2),
                    (X["Rating"] == "Strongly Disagree", 1)
                )
                )
)

survey_gather

/home/sandrews/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/dfply/vector.py:110: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  return coalescer.lookup(np.arange(coalescer.shape[0]), min_nonna)


,User,Response,Rating,Lickert
0,1,F1,Somewhat Agree,4.0
1,2,F1,Somewhat Agree,4.0
2,3,F1,Strongly Agree,5.0
3,4,F1,Somewhat Agree,4.0
4,5,F1,Strongly Agree,5.0
...,...,...,...,...
11611,260,F2.11,Neither Agree nor Disagree,3.0
11612,261,F2.11,Somewhat Agree,4.0
11613,262,F2.11,Somewhat Agree,4.0
11614,263,F2.11,Somewhat Agree,4.0


In [50]:
survey_spread = (survey_gather
>> drop(X["Rating"])
>> spread(X.Response, X.Lickert)

)
survey_spread


,User,F1,F1.1,F1.2,F1.3,F1.4,F1.5,F1.6,F1.7,F2,...,F5.3,F5.4,F5.5,F5.6,F5.7,F6,F6.1,F6.2,F6.3,F6.4
0,1,4.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,...,4.0,4.0,2.0,4.0,4.0,2.0,4.0,4.0,2.0,4.0
1,2,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,...,4.0,3.0,3.0,2.0,4.0,2.0,4.0,4.0,2.0,2.0
2,3,5.0,5.0,5.0,5.0,2.0,5.0,5.0,4.0,4.0,...,4.0,4.0,2.0,2.0,5.0,2.0,3.0,4.0,2.0,4.0
3,4,4.0,4.0,5.0,4.0,3.0,4.0,5.0,3.0,5.0,...,4.0,4.0,2.0,2.0,4.0,3.0,4.0,4.0,4.0,3.0
4,5,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,3.0,...,5.0,3.0,1.0,1.0,5.0,1.0,4.0,5.0,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,260,4.0,3.0,5.0,4.0,4.0,4.0,5.0,3.0,4.0,...,4.0,4.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0,3.0
260,261,4.0,5.0,4.0,4.0,3.0,4.0,4.0,3.0,5.0,...,4.0,4.0,2.0,3.0,4.0,2.0,4.0,3.0,2.0,NaN
261,262,4.0,5.0,5.0,3.0,2.0,4.0,5.0,2.0,4.0,...,4.0,4.0,1.0,1.0,3.0,2.0,4.0,4.0,2.0,3.0
262,263,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,...,2.0,2.0,2.0,1.0,5.0,2.0,4.0,5.0,2.0,2.0


In [51]:
#all in one pipe


survey_adjusted = (survey
                >> rename(User = X["Unnamed: 0"])
                >> gather("Response","Rating", survey_cols)
                >> mutate(Lickert = case_when(
                    (X["Rating"] == "Strongly Agree", 5),
                    (X["Rating"] == "Somewhat Agree", 4),
                    (X["Rating"] == "Neither Agree nor Disagree", 3),
                    (X["Rating"] == "Somewhat Disagree", 2),
                    (X["Rating"] == "Strongly Disagree", 1)
                )
                )
                >> drop(X["Rating"])
                >> spread(X.Response, X.Lickert)

)

survey_adjusted

/home/sandrews/.pyenv/versions/anaconda3-2020.02/lib/python3.7/site-packages/dfply/vector.py:110: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  return coalescer.lookup(np.arange(coalescer.shape[0]), min_nonna)


,User,F1,F1.1,F1.2,F1.3,F1.4,F1.5,F1.6,F1.7,F2,...,F5.3,F5.4,F5.5,F5.6,F5.7,F6,F6.1,F6.2,F6.3,F6.4
0,1,4.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,4.0,...,4.0,4.0,2.0,4.0,4.0,2.0,4.0,4.0,2.0,4.0
1,2,4.0,4.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,...,4.0,3.0,3.0,2.0,4.0,2.0,4.0,4.0,2.0,2.0
2,3,5.0,5.0,5.0,5.0,2.0,5.0,5.0,4.0,4.0,...,4.0,4.0,2.0,2.0,5.0,2.0,3.0,4.0,2.0,4.0
3,4,4.0,4.0,5.0,4.0,3.0,4.0,5.0,3.0,5.0,...,4.0,4.0,2.0,2.0,4.0,3.0,4.0,4.0,4.0,3.0
4,5,5.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,3.0,...,5.0,3.0,1.0,1.0,5.0,1.0,4.0,5.0,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,260,4.0,3.0,5.0,4.0,4.0,4.0,5.0,3.0,4.0,...,4.0,4.0,1.0,1.0,4.0,1.0,4.0,5.0,1.0,3.0
260,261,4.0,5.0,4.0,4.0,3.0,4.0,4.0,3.0,5.0,...,4.0,4.0,2.0,3.0,4.0,2.0,4.0,3.0,2.0,NaN
261,262,4.0,5.0,5.0,3.0,2.0,4.0,5.0,2.0,4.0,...,4.0,4.0,1.0,1.0,3.0,2.0,4.0,4.0,2.0,3.0
262,263,5.0,5.0,5.0,5.0,4.0,5.0,5.0,4.0,5.0,...,2.0,2.0,2.0,1.0,5.0,2.0,4.0,5.0,2.0,2.0
